This workbook runs a process whereby an starting position (the opening position with White to move for now) is taken as an input.  It then builds a Python array with ALL the possible moves for a defined level of half-moves (i.e. 0 - 6).  The array generates the leaf nodes of the tree that represents the starting positions for a random play through to conclusion.  

DONE:
-Once the leaf node (starting position) is selected, random moves will be played through to critical state (DRAW or WIN).  
-The idea is then to propagate the reward back up the tree to the parent nodes in the game.
-Then one could aggregate the reward for each one of the branches down to the starting position and pick a next move based on the optimal reward (largest for White, smallest for Black).

TO DO: 
-Lastly, the game tree could be transferred to a DataFrame and persisted to the Teradata database.

CHALLENGES:
-The tree explodes exponentially, and anything past a 2 half-move depth becomes prohibitively slow.

In [1]:
import chess
import random
import time

random.seed(time.time())

First we create some helper functions to work with the Python-chess library.

In [2]:
def legal_moves_array(in_board):
        # Gets all the legal moves for the board and enumerate them in an array.
        all_moves = str(in_board.legal_moves)
        start_pos = [pos for pos, char in enumerate(all_moves) if char == '(']
        end_pos = [pos for pos, char in enumerate(all_moves) if char == ')']
        all_moves_string= all_moves[start_pos[0]+1:end_pos[0]]
        all_moves_string = all_moves_string.replace(" ", "")
        all_moves_array = all_moves_string.split(',')
        
        return all_moves_array

def truncate_fen(in_board):
        # Truncates the FEN to exclude the ply and full move.
        fen = in_board.fen()
        spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
        truncated_fen = fen[:spaces_pos[3]]
        
        return truncated_fen

def play_leaf(in_FEN, discount=0.95):
        half_moves = 0
        leaf_board = chess.Board()
        leaf_board.set_fen(in_FEN)
        while not leaf_board.is_checkmate() and not leaf_board.is_stalemate() and not leaf_board.is_insufficient_material() and not leaf_board.can_claim_draw() and not leaf_board.is_seventyfive_moves() and not leaf_board.is_fivefold_repetition():
                all_moves = str(leaf_board.legal_moves)
                start_pos = [pos for pos, char in enumerate(all_moves) if char == '(']
                end_pos = [pos for pos, char in enumerate(all_moves) if char == ')']
                all_moves_string= all_moves[start_pos[0]+1:end_pos[0]]
                all_moves_string = all_moves_string.replace(" ", "")
                all_moves_array = all_moves_string.split(',')
                next_move = random.choice(all_moves_array)
                leaf_board.push_san(next_move)
                half_moves += 1
        
        if leaf_board.is_checkmate():
                if leaf_board.outcome().winner:
                        reward = 1
                else:
                        reward = -1
        else:
                reward = 0

        leaf_reward = reward*discount**(half_moves)
        
        return leaf_reward

def update_parent_nodes(in_batch_arrary, depth):
        for row in reversed(in_batch_arrary):

                parent_node = row[2]
                children = 0
                children_total_reward = 0
                for child_row in in_batch_arrary:
                        if child_row[2] == parent_node:
                                children = children + 1
                                children_total_reward = children_total_reward + child_row[5]
                                children_average_reward = children_total_reward / children
                in_batch_arrary[row[2]][5] = children_average_reward

        batch_array = in_batch_arrary
        return batch_array
        

The next section builds the Monte Carlo Tree from a starting position (in this case the starting position is the opening position with White making the first move).  Tthe depth of the search can be specified using the parameter depth = n.  Just note that the depth and legal variations from the opening position are shown respectively below so stay away from depth > 3'ish.

(1:20), (2:400), (3:8902), (4:197281), (5:4865609), (6:119060324) 

The leaf nodes (boards) are written to the array batch_array.  The leaf nodes of the batch_array represents the starting point of the games to be played.

In [3]:
depth = 3
leafs_played = 0
node_board = chess.Board()

# This is optional in-case you want to start the play at a specific FEN, otherwise it will start from the opening position.
node_board.set_fen('rnbqkbnr/pppppppp/8/8/5P2/8/PPPPP1PP/RNBQKBNR b KQkq -')

truncated_fen = truncate_fen(node_board)
node_index = 0

test_fen = ''

# Initialize the game_tree data.
batch_array = [
    # Node Index, Frame Level, Parent Move Index, Truncated FEN, Move, Reward
    [node_index,0,0,truncated_fen,'',0.]
]

control_array = batch_array


for d_level in range(depth):
    frame_level = d_level + 1

    for row in control_array:
        
        if row[1] == d_level:
            node_board.set_fen(row[3])
            if not node_board.is_checkmate() and not node_board.is_stalemate() and not node_board.is_insufficient_material() and not node_board.can_claim_draw() and not node_board.is_seventyfive_moves() and not node_board.is_fivefold_repetition():
                legal_moves = legal_moves_array(node_board)
                
                for move in legal_moves:
                    node_board.push_san(move)
                    truncated_fen = truncate_fen(node_board)
                    node_index = node_index + 1
                    
                    if d_level+1 == depth:
                        # Play the leaf
                        leaf_reward = play_leaf(truncated_fen)
                        leafs_played = leafs_played + 1
                        print("Leafs played: " + str(leafs_played))
                        half_move_array = [node_index, frame_level, row[0],truncated_fen, move, leaf_reward]
                        batch_array.append(half_move_array)
                    else:
                        half_move_array = [node_index, frame_level, row[0],truncated_fen, move, 0]
                        batch_array.append(half_move_array)                       
                    
                    node_board.pop()

    control_array = batch_array

# Now update the parent nodes with the sum of the leaf node rewards.
batch_array = update_parent_nodes(batch_array, depth)     

KeyboardInterrupt: 

In [22]:
#counter = 0
#print(len(batch_array))
#for row in batch_array:
#    if row[1] == depth:
#        counter += 1
#        print(row)

#print(counter)

In [23]:
max_reward = 0
min_reward = 0
best_move_black = ''
best_move_white = ''

counter = 0
for row in batch_array:
   if row[1]==1:
        if max_reward <= row[5]:
            max_reward = row[5]
            best_move_white = row[4]
        elif min_reward >= row[5]:
            min_reward = row[5]
            best_move_black = row[4]

print(best_move_white, best_move_black)


c6 e5


In [24]:
for row in batch_array:
    print(row)

[0, 0, 0, 'rnbqkbnr/pppppppp/8/8/5P2/8/PPPPP1PP/RNBQKBNR b KQkq -', '', 0.000634367524743435]
[1, 1, 0, 'rnbqkb1r/pppppppp/7n/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'Nh6', 0.0002575846441545322]
[2, 1, 0, 'rnbqkb1r/pppppppp/5n2/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'Nf6', 0.0031800176441505604]
[3, 1, 0, 'r1bqkbnr/pppppppp/2n5/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'Nc6', 0.00031592392218111854]
[4, 1, 0, 'r1bqkbnr/pppppppp/n7/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'Na6', 0.00026251934262758826]
[5, 1, 0, 'rnbqkbnr/ppppppp1/7p/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'h6', -0.0001865716456355278]
[6, 1, 0, 'rnbqkbnr/pppppp1p/6p1/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'g6', 0.0003014323243793266]
[7, 1, 0, 'rnbqkbnr/ppppp1pp/5p2/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'f6', -6.688289861792271e-05]
[8, 1, 0, 'rnbqkbnr/pppp1ppp/4p3/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'e6', -0.00010944386715777847]
[9, 1, 0, 'rnbqkbnr/ppp1pppp/3p4/8/5P2/8/PPPPP1PP/RNBQKBNR w KQkq -', 'd6', -0.0009481268753808317]
[10, 1,